In [26]:
pip install numpy -q

In [27]:
pip install pandas -q

In [28]:
pip install scikit-learn -q

In [38]:
import seaborn as sns
df = sns.load_dataset('iris')
df.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')

In [39]:
import numpy as np
from sklearn.metrics import adjusted_rand_score
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample

df["species"].replace({"setosa": 0, "versicolor": 1, "virginica": 2}, inplace=True)

# Renomeando as colunas
df.columns = ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm", "Class"]

# Separando os labels (classes)
labels = df["Class"].values
df.drop("Class", axis=1, inplace=True)

# Normalizando os dados via Min-Max
scaler = MinMaxScaler()
dados = scaler.fit_transform(df).astype(np.float64)

# Fuzzyficar os labels
def fuzzyficar_labels(labels, num_clusters):
    fuzzy_labels = np.zeros((len(labels), num_clusters))
    for i, label in enumerate(labels):
        fuzzy_labels[i, label] = 1
    return fuzzy_labels

# Funções para FCM, corrigidas conforme as fórmulas fornecidas

# Função de distância Euclidiana
def distancia_euclidiana(x_k, y_j):
    return np.sum((x_k - y_j) ** 2)

# Inicialização da matriz de pertinência aleatória
def inicializao_matriz_pertinencia(num_amostras, num_clusters):
    matriz_pertinencia = np.random.rand(num_amostras, num_clusters)
    matriz_pertinencia = matriz_pertinencia / matriz_pertinencia.sum(axis=1, keepdims=True)
    return matriz_pertinencia

# Atualização dos centroides usando a fórmula corrigida (equação 2.3)
def atualizacao_centroides(dados, matriz_pertinencia, m):
    matriz_pertinencia_m = matriz_pertinencia ** m
    num = np.dot(matriz_pertinencia_m.T, dados)
    den = np.sum(matriz_pertinencia_m.T, axis=1, keepdims=True)
    centroides = num / den
    return centroides

# Atualização da matriz de pertinência (equação 2.4)
def atualizacao_matriz_pertinencia(dados, centroides, m):
    num_clusters = centroides.shape[0]
    matriz_pertinencia = np.zeros((dados.shape[0], num_clusters))

    for k in range(dados.shape[0]):
        for i in range(num_clusters):
            dist_ki = distancia_euclidiana(dados[k], centroides[i])
            sum_ratio = 0
            for j in range(num_clusters):
                dist_kj = distancia_euclidiana(dados[k], centroides[j])
                sum_ratio += (dist_ki / dist_kj) ** (2 / (m - 1))
            matriz_pertinencia[k, i] = 1 / sum_ratio

    return matriz_pertinencia

# Função principal do FCM
def fcm(dados, num_clusters, m=2, max_iter=1000, erro=1e-5):
    num_amostras = dados.shape[0]
    matriz_pertinencia = inicializao_matriz_pertinencia(num_amostras, num_clusters)

    for _ in range(max_iter):
        centroides = atualizacao_centroides(dados, matriz_pertinencia, m)
        nova_matriz_pertinencia = atualizacao_matriz_pertinencia(dados, centroides, m)

        if np.linalg.norm(nova_matriz_pertinencia - matriz_pertinencia) < erro:
            break

        matriz_pertinencia = nova_matriz_pertinencia

    return centroides, matriz_pertinencia

# Função de similaridade fuzzy
def similaridade_fuzzy(pertinencia_k, pertinencia_l):
    return 1 - np.linalg.norm(pertinencia_k - pertinencia_l)**2

# Índice de Rand Difuso
def indice_rand_difuso(labels_fuzzy, matriz_pertinencia):
    n = labels_fuzzy.shape[0]
    somatorio_diferencas = 0
    num_total_pares = n * (n - 1) / 2

    for k in range(n):
        for l in range(k + 1, n):
            EP = similaridade_fuzzy(labels_fuzzy[k], labels_fuzzy[l])
            EQ = similaridade_fuzzy(matriz_pertinencia[k], matriz_pertinencia[l])
            somatorio_diferencas += abs(EP - EQ)

    FR = 1 - (somatorio_diferencas / num_total_pares)
    return FR

# Implementação do Bootstrap
def bootstrap_experimento(dados, labels_fuzzy, num_clusters, num_trials, num_execucoes=200):
    melhores_indices_rand = []

    for trial in range(num_trials):
        dados_resample, labels_resample = resample(dados, labels_fuzzy)

        melhor_indice_rand = -np.inf

        for _ in range(num_execucoes):
            centroides, matriz_pertinencia = fcm(dados_resample, num_clusters)
            idx_rand_difuso = indice_rand_difuso(labels_resample, matriz_pertinencia)

            if idx_rand_difuso > melhor_indice_rand:
                melhor_indice_rand = idx_rand_difuso

        melhores_indices_rand.append(melhor_indice_rand)
        print(f"Trial {trial} - Índice Rand: {melhor_indice_rand}")

    mean_rand_index = np.mean(melhores_indices_rand)
    std_rand_index = np.std(melhores_indices_rand)
    return mean_rand_index, std_rand_index

# Fuzzyficando os labels
labels_fuzzy = fuzzyficar_labels(labels, num_clusters=3)

# Parâmetros do experimento
num_clusters = 3
num_trials = 1000

# Executando o Bootstrap
media_indice_rand, dp_indice_rand = bootstrap_experimento(dados, labels_fuzzy, num_clusters, num_trials)

print(f"Bootstrap FCM Clustering Results ({num_trials} reamostragens)")
print(f"Mean Fuzzy Rand Index: {media_indice_rand:.4f}")
print(f"Standard Deviation of Fuzzy Rand Index: {dp_indice_rand:.4f}")

<ipython-input-39-e277cb62eff8>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["species"].replace({"setosa": 0, "versicolor": 1, "virginica": 2}, inplace=True)
<ipython-input-39-e277cb62eff8>:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["species"].replace({"setosa": 0, "versicolor": 1, "

KeyboardInterrupt: 